In [2]:
from __future__ import print_function

import sys
import os
import pandas as pd
import numpy as np
import re
import nltk

from keras.layers import Input, Embedding, LSTM, TimeDistributed, Dense, Bidirectional
from keras.models import Model, load_model

INPUT_LENGTH = 20
OUTPUT_LENGTH = 20

Using TensorFlow backend.


In [3]:
lines = open('movie_lines.txt',encoding = 'utf-8',errors = 'ignore').read().split('\n')

In [4]:
conv_lines = open('movie_conversations.txt',encoding = 'utf-8',errors = 'ignore').read().split('\n')
# conv_lines

In [5]:
# Create a dictionary to map each line's id with its text
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [6]:
# Create a list of all of the conversations' lines' ids.
convs = []
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    convs.append(_line.split(','))

In [7]:
#id and conversation sample
for k in convs[400]:
    print (k, id2line[k])

L2563 Next.  Could I see your documents, please?
L2564 Yes sir.


In [8]:
# Sort the sentences into questions (inputs) and answers (targets)
questions = []
answers = []
for conv in convs:
    for i in range(len(conv)-1):
        questions.append(id2line[conv[i]])
        answers.append(id2line[conv[i+1]])
        
# Compare lengths of questions and answers
print(len(questions))
print(len(answers))

221616
221616


In [10]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|]", "", text)
#     text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = " ".join(text.split())
    return text

In [11]:
# Clean the data
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))
clean_answers = []    
for answer in answers:
    clean_answers.append(clean_text(answer))

In [12]:
# Find the length of sentences (not using nltk due to processing speed)
lengths = []
# lengths.append([len(nltk.word_tokenize(sent)) for sent in clean_questions]) #nltk approach
for question in clean_questions:
    lengths.append(len(question.split()))
for answer in clean_answers:
    lengths.append(len(answer.split()))
# Create a dataframe so that the values can be inspected
lengths = pd.DataFrame(lengths, columns=['counts'])
print(np.percentile(lengths, 80))
print(np.percentile(lengths, 85))
print(np.percentile(lengths, 90))
print(np.percentile(lengths, 95))

16.0
19.0
24.0
32.0


In [13]:
# Remove questions and answers that are shorter than 1 word and longer than 20 words.
min_line_length = 2
max_line_length = 20

# Filter out the questions that are too short/long
short_questions_temp = []
short_answers_temp = []

for i, question in enumerate(clean_questions):
    if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length:
        short_questions_temp.append(question)
        short_answers_temp.append(clean_answers[i])

# Filter out the answers that are too short/long
short_questions = []
short_answers = []

for i, answer in enumerate(short_answers_temp):
    if len(answer.split()) >= min_line_length and len(answer.split()) <= max_line_length:
        short_answers.append(answer)
        short_questions.append(short_questions_temp[i])
        
print(len(short_questions))
print(len(short_answers))

138528
138528


In [14]:
r = np.random.randint(1,len(short_questions))

for i in range(r, r+3):
    print(short_questions[i])
    print(short_answers[i])
    print()

then how did you get here?
i always eat here.

i always eat here.
this is a place for workmen.

just an old man. his memory is getting weak.
what are you after?



In [15]:
nltk.download('punkt') # first-time use only
nltk.download('wordnet') # first-time use only

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Niyati\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Niyati\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
#choosing number of samples
num_samples = 30000  # Number of samples to train on.
short_questions = short_questions[:num_samples]
short_answers = short_answers[:num_samples]
#tokenizing the qns and answers
short_questions_tok = [nltk.word_tokenize(sent) for sent in short_questions]
short_answers_tok = [nltk.word_tokenize(sent) for sent in short_answers]

In [17]:
#train-validation split
data_size = len(short_questions_tok)

# We will use the first 0-80th %-tile (80%) of data for the training
training_input  = short_questions_tok[:round(data_size*(80/100))]
training_input  = [tr_input[::-1] for tr_input in training_input] #reverseing input seq for better performance
training_output = short_answers_tok[:round(data_size*(80/100))]

# We will use the remaining for validation
validation_input = short_questions_tok[round(data_size*(80/100)):]
validation_input  = [val_input[::-1] for val_input in validation_input] #reverseing input seq for better performance
validation_output = short_answers_tok[round(data_size*(80/100)):]

print('training size', len(training_input))
print('validation size', len(validation_input))

training size 24000
validation size 6000


In [18]:
# Create a dictionary for the frequency of the vocabulary
# Create 
vocab = {}
for question in short_questions_tok:
    for word in question:
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

for answer in short_answers_tok:
    for word in answer:
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1         

In [19]:
# Remove rare words from the vocabulary.
# We will aim to replace fewer than 5% of words with <UNK>
# You will see this ratio soon.
threshold = 15
count = 0
for k,v in vocab.items():
    if v >= threshold:
        count += 1

In [20]:
print("Size of total vocab:", len(vocab))
print("Size of vocab we will use:", count)

Size of total vocab: 16382
Size of vocab we will use: 1946


In [21]:
#we will create dictionaries to provide a unique integer for each word.
WORD_CODE_START = 1
WORD_CODE_PADDING = 0


word_num  = 2 #number 1 is left for WORD_CODE_START for model decoder later
encoding = {}
decoding = {1: 'START'}
for word, count in vocab.items():
    if count >= threshold: #get vocabularies that appear above threshold count
        encoding[word] = word_num 
        decoding[word_num ] = word
        word_num += 1

print("No. of vocab used:", word_num)

No. of vocab used: 1948


In [22]:
#include unknown token for words not in dictionary
decoding[len(encoding)+2] = '<UNK>'
encoding['<UNK>'] = len(encoding)+2

In [23]:
dict_size = word_num+1
dict_size

1949

In [24]:
def transform(encoding, data, vector_size=20):
    """
    :param encoding: encoding dict built by build_word_encoding()
    :param data: list of strings
    :param vector_size: size of each encoded vector
    """
    transformed_data = np.zeros(shape=(len(data), vector_size))
    for i in range(len(data)):
        for j in range(min(len(data[i]), vector_size)):
            try:
                transformed_data[i][j] = encoding[data[i][j]]
            except:
                transformed_data[i][j] = encoding['<UNK>']
    return transformed_data


In [25]:
#encoding training set
encoded_training_input = transform(
    encoding, training_input, vector_size=INPUT_LENGTH)
encoded_training_output = transform(
    encoding, training_output, vector_size=OUTPUT_LENGTH)

print('encoded_training_input', encoded_training_input.shape)
print('encoded_training_output', encoded_training_output.shape)

encoded_training_input (24000, 20)
encoded_training_output (24000, 20)


In [26]:
#encoding validation set
encoded_validation_input = transform(
    encoding, validation_input, vector_size=INPUT_LENGTH)
encoded_validation_output = transform(
    encoding, validation_output, vector_size=OUTPUT_LENGTH)

print('encoded_validation_input', encoded_validation_input.shape)
print('encoded_validation_output', encoded_validation_output.shape)

encoded_validation_input (6000, 20)
encoded_validation_output (6000, 20)


In [27]:
import tensorflow as tf
tf.keras.backend.clear_session()
tf.autograph.set_verbosity(0)
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [28]:
INPUT_LENGTH = 20
OUTPUT_LENGTH = 20

encoder_input = Input(shape=(INPUT_LENGTH,))
decoder_input = Input(shape=(OUTPUT_LENGTH,))


In [29]:
from keras.layers import SimpleRNN

encoder = Embedding(dict_size, 128, input_length=INPUT_LENGTH, mask_zero=True)(encoder_input)
encoder = LSTM(512, return_sequences=True, unroll=True)(encoder)
encoder_last = encoder[:,-1,:]

print('encoder', encoder)
print('encoder_last', encoder_last)

decoder = Embedding(dict_size, 128, input_length=OUTPUT_LENGTH, mask_zero=True)(decoder_input)
decoder = LSTM(512, return_sequences=True, unroll=True)(decoder, initial_state=[encoder_last, encoder_last])

print('decoder', decoder)

# For the plain Sequence-to-Sequence, we produced the output from directly from decoder
# output = TimeDistributed(Dense(output_dict_size, activation="softmax"))(decoder)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
encoder Tensor("lstm_1/transpose_2:0", shape=(?, 20, 512), dtype=float32)
encoder_last Tensor("strided_slice:0", shape=(?, 512), dtype=float32)
decoder Tensor("lstm_2/transpose_2:0", shape=(?, 20, 512), dtype=float32)


In [30]:
from keras.layers import Activation, dot, concatenate

attention = dot([decoder, encoder], axes=[2, 2])
attention = Activation('softmax', name='attention')(attention)
print('attention', attention)

context = dot([attention, encoder], axes=[2,1])
print('context', context)

decoder_combined_context = concatenate([context, decoder])
print('decoder_combined_context', decoder_combined_context)

# Has another weight + tanh layer as described in equation (5) of the paper
output = TimeDistributed(Dense(512, activation="tanh"))(decoder_combined_context)
output = TimeDistributed(Dense(dict_size, activation="softmax"))(output)
print('output', output)

attention Tensor("attention/truediv:0", shape=(?, 20, 20), dtype=float32)
context Tensor("dot_2/MatMul:0", shape=(?, 20, 512), dtype=float32)
decoder_combined_context Tensor("concatenate_1/concat:0", shape=(?, 20, 1024), dtype=float32)
output Tensor("time_distributed_2/Reshape_1:0", shape=(?, 20, 1949), dtype=float32)


In [31]:
model = Model(inputs=[encoder_input, decoder_input], outputs=[output])
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 20, 128)      249472      input_2[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 20, 128)      249472      input_1[0][0]                    
____________________________________________________________________________________________

In [ ]:
model.layers
len(model.layers)

In [ ]:
model.summary()

In [32]:
training_encoder_input = encoded_training_input
training_decoder_input = np.zeros_like(encoded_training_output)
training_decoder_input[:, 1:] = encoded_training_output[:,:-1]
training_decoder_input[:, 0] = WORD_CODE_START
training_decoder_output = np.eye(dict_size)[encoded_training_output.astype('int')]

validation_encoder_input = encoded_validation_input
validation_decoder_input = np.zeros_like(encoded_validation_output)
validation_decoder_input[:, 1:] = encoded_validation_output[:,:-1]
validation_decoder_input[:, 0] = WORD_CODE_START
validation_decoder_output = np.eye(dict_size)[encoded_validation_output.astype('int')]

In [33]:
# model_save_name = 'classifier.pt'
# path = F"/content/gdrive/My Drive/{model_save_name}" 
# torch.save(model.state_dict(), path)

In [34]:
model.fit(x=[training_encoder_input, training_decoder_input], y=[training_decoder_output],
          validation_data=([validation_encoder_input, validation_decoder_input], [validation_decoder_output]),
          validation_split=0.05,
          batch_size=240, epochs=500)

from keras.models import load_model

model.save('model_atention1.h5')  # creates a HDF5 file 'my_model.h5'


Train on 24000 samples, validate on 6000 samples
Epoch 1/500
24000/24000 [==============================] - 72s 3ms/step - loss: 0.0020 - val_loss: 0.0017
Epoch 2/500
24000/24000 [==============================] - 19s 783us/step - loss: 0.0016 - val_loss: 0.0016
Epoch 3/500
24000/24000 [==============================] - 19s 786us/step - loss: 0.0016 - val_loss: 0.0016
Epoch 4/500
24000/24000 [==============================] - 19s 787us/step - loss: 0.0015 - val_loss: 0.0015
Epoch 5/500
24000/24000 [==============================] - 19s 786us/step - loss: 0.0015 - val_loss: 0.0015
Epoch 6/500
24000/24000 [==============================] - 19s 790us/step - loss: 0.0015 - val_loss: 0.0015
Epoch 7/500
24000/24000 [==============================] - 19s 808us/step - loss: 0.0014 - val_loss: 0.0014
Epoch 8/500
24000/24000 [==============================] - 20s 832us/step - loss: 0.0014 - val_loss: 0.0014
Epoch 9/500
24000/24000 [==============================] - 20s 839us/step - loss: 0.0014

24000/24000 [==============================] - 21s 882us/step - loss: 3.2555e-04 - val_loss: 0.0022
Epoch 144/500
24000/24000 [==============================] - 24s 990us/step - loss: 3.2006e-04 - val_loss: 0.0022
Epoch 145/500
24000/24000 [==============================] - 26s 1ms/step - loss: 3.1570e-04 - val_loss: 0.0022
Epoch 146/500
24000/24000 [==============================] - 25s 1ms/step - loss: 3.1650e-04 - val_loss: 0.0022
Epoch 147/500
24000/24000 [==============================] - 25s 1ms/step - loss: 3.2451e-04 - val_loss: 0.0022
Epoch 148/500
24000/24000 [==============================] - 34s 1ms/step - loss: 3.8405e-04 - val_loss: 0.0022
Epoch 149/500
24000/24000 [==============================] - 23s 947us/step - loss: 4.2368e-04 - val_loss: 0.0022
Epoch 150/500
24000/24000 [==============================] - 23s 947us/step - loss: 3.8760e-04 - val_loss: 0.0022
Epoch 151/500
24000/24000 [==============================] - 31s 1ms/step - loss: 3.4171e-04 - val_loss: 0.002

Epoch 216/500
24000/24000 [==============================] - 21s 866us/step - loss: 5.8957e-05 - val_loss: 0.0027
Epoch 217/500
24000/24000 [==============================] - 24s 997us/step - loss: 5.7064e-05 - val_loss: 0.0027
Epoch 218/500
24000/24000 [==============================] - 36s 2ms/step - loss: 5.5316e-05 - val_loss: 0.0027
Epoch 219/500
24000/24000 [==============================] - 21s 860us/step - loss: 5.6122e-05 - val_loss: 0.0027
Epoch 220/500
24000/24000 [==============================] - 32s 1ms/step - loss: 6.9881e-05 - val_loss: 0.0027
Epoch 221/500
24000/24000 [==============================] - 29s 1ms/step - loss: 3.1744e-04 - val_loss: 0.0027
Epoch 222/500
24000/24000 [==============================] - 21s 860us/step - loss: 3.5165e-04 - val_loss: 0.0026
Epoch 223/500
24000/24000 [==============================] - 21s 858us/step - loss: 2.2957e-04 - val_loss: 0.00260
Epoch 224/500
24000/24000 [==============================] - 21s 886us/step - loss: 1.5703e-0

24000/24000 [==============================] - 25s 1ms/step - loss: 1.2655e-05 - val_loss: 0.0029
Epoch 361/500
24000/24000 [==============================] - 21s 876us/step - loss: 1.2615e-05 - val_loss: 0.0029
Epoch 362/500
24000/24000 [==============================] - 22s 928us/step - loss: 1.2486e-05 - val_loss: 0.0029
Epoch 363/500
24000/24000 [==============================] - 26s 1ms/step - loss: 1.2505e-05 - val_loss: 0.0029
Epoch 364/500
24000/24000 [==============================] - 25s 1ms/step - loss: 1.2436e-05 - val_loss: 0.0029
Epoch 365/500
24000/24000 [==============================] - 30s 1ms/step - loss: 1.2475e-05 - val_loss: 0.0029
Epoch 366/500
24000/24000 [==============================] - 28s 1ms/step - loss: 1.2370e-05 - val_loss: 0.0029
Epoch 367/500
24000/24000 [==============================] - 22s 913us/step - loss: 1.2341e-05 - val_loss: 0.0029
Epoch 368/500
24000/24000 [==============================] - 22s 926us/step - loss: 1.2281e-05 - val_loss: 0.002

c:\users\niyati\anaconda3\envs\tf-gpu\lib\site-packages\keras\engine\network.py:896: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'strided_slice:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'strided_slice:0' shape=(?, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [35]:
from keras.models import load_model
# returns a compiled model
# identical to the previous one
model = load_model('model_atention1.h5')

In [36]:
def prediction(raw_input):
    clean_input = clean_text(raw_input)
    input_tok = [nltk.word_tokenize(clean_input)]
    input_tok = [input_tok[0][::-1]]  #reverseing input seq
    encoder_input = transform(encoding, input_tok, 20)
    decoder_input = np.zeros(shape=(len(encoder_input), OUTPUT_LENGTH))
    decoder_input[:,0] = WORD_CODE_START
    for i in range(1, OUTPUT_LENGTH):
        output = model.predict([encoder_input, decoder_input]).argmax(axis=2)
        decoder_input[:,i] = output[:,i]
    return output

def decode(decoding, vector):
    """
    :param decoding: decoding dict built by word encoding
    :param vector: an encoded vector
    """
    text = ''
    for i in vector:
        if i == 0:
            break
        text += ' '
        text += decoding[i]
    return text

In [37]:
for i in range(20):
    seq_index = np.random.randint(1, len(short_questions))
    output = prediction(short_questions[seq_index])
    print ('bot:', short_questions[seq_index])
    print ('human:', decode(decoding, output[0]))

bot: a faerie's love makes anything possible.
human:  what have you got ?
bot: i could not sleep... i should get going i feel really weird...
human:  what about ?
bot: my car's in the garage.
human:  what i mean is , do not make any of <UNK> for <UNK> <UNK> .
bot: where are we, mean man?
human:  who <UNK> it ?
bot: y'know my lady lucindy?
human:  who am the papers ?
bot: but you are old!
human:  who does he
bot: you must be edward bloom.
human:  <UNK> .
bot: thanks for playing along. i just have to sit for a while.
human:  what are you saying ?
bot: a naked woman, chained in ya house?
human:  what ...
bot: people show their happiness in a lot of different ways.
human:  what have you think ?
bot: okay, how do you want to do it?
human:  what about it ?
bot: uhuh. i ai not riding in that trunk no minutes. why do not i just ride with you?
human:  who needs to be playing ?
bot: do you dream?
human:  what happened ?
bot: and how long have you been sleeping with mrs. windham?
human:  what hap

In [ ]:
while True:
  raw_input = input()
  output = prediction(raw_input)
  print (decode(decoding, output[0]))

hey
 what i mean .
what
 what i mean .
mean
 what i mean .
hey
 what i mean .
how are u
 <UNK> hours into <UNK> .
into what
 what i mean .
overtrain
 what i mean .
haha
 what i mean .
overtrained model
 who would their money ? <UNK> inside him !
whats inside him
 what i mean .
